In [3]:
%pip install -qU pypdf

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'e:\Programming-Projects\Hackathon\hackrx6\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
from dotenv import load_dotenv
from langchain import hub
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

from utilities.pdf_utils import load_pdf_using_PyPDF
from utilities.text_utils import format_docs, log_chunks

In [2]:
file_path = "docs/BAJHLIP23020V012223.pdf"
pages = await load_pdf_using_PyPDF(file_path) # Average time with PyPDF: 2.5s

In [3]:
# Splitting into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

In [4]:
# Embeddings
all_mini_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
bge_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

embeddings = bge_embeddings

e:\Programming-Projects\Hackathon\hackrx6\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Creating Vector Store
vector_store = FAISS.from_documents(splits, embedding=embeddings)

In [ ]:
# Store the vector DB locally to save processing time
vector_store.save_local("vector_store/faiss_index_bajaj")

In [5]:
# Load the saved vector store
vector_store = FAISS.load_local("vector_store/faiss_index_bajaj", embeddings, allow_dangerous_deserialization=True)

In [6]:
retriever = vector_store.as_retriever()

In [7]:
load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [8]:
# Retrieve, Log and format the chunks
logged_retriever = retriever | RunnableLambda(log_chunks) | format_docs

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

# Chain
rag_chain = (
    {"context": logged_retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
# rag_chain.invoke("what is the minimum and maximum In-patient Hospitalization Treatment Limits for Imperial plus plan?")
# rag_chain.invoke("what was the last question i asked?") # No Memory of last question
# rag_chain.invoke("46-year-old male, knee surgery in Pune, 3-month-old insurance policy")
# rag_chain.invoke("knee surgery?")


Retrieved Chunks:
Policy Period. 
iv. At the end of the Policy Period, the Policy shall terminate and can be renewed within the Grace Period of 30 
days to maintain continuity of benefits without break in Policy. Coverage is not available during the grace 
period. 
v. No loading shall apply on renewals based on individual claims experience 
 
11. Withdrawal of Policy 
i. In the likelihood of this product being withdrawn in future, the Company will intimate the insured person about 
the same 90 days prior to expiry of the Policy. 
ii. Insured Person will have the option to migrate to similar health insurance product available with the Company at 
the time of renewal with all the accrued continuity benefits such as cumulative bonus, waiver of waiting period 
as per IRDAI guidelines, provided the Policy has been maintained without a break. 
 
12. Moratorium Period 
After completion of eight continuous years under this Policy no look back to be applied. This period of eight years is
---
o

"I'm sorry, but the context provided does not contain information about your previous questions. Therefore, I am unable to answer your question."